# Demo of parsing Markdown nicely

## Imports

In [3]:
# Loading and Saving files & others
import pathlib
import sys
import numpy as np

# Parse Markdown
from markdown_it import MarkdownIt # pip install markdown-it-py 
from mdformat.renderer import MDRenderer # pip install mdformat

# Dealing with YAML
import yaml

# deal with multi-line strings in YAML Dump
## Code copied from here: https://stackoverflow.com/a/33300001/2217577

def str_presenter(dumper, data):
    if len(data.splitlines()) > 1:  # check for multiline string
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

## Read files

In [6]:
# Read the markdown file 

path = 'q01_multiple-choice/q01_multiple-choice.md'
mdtext = pathlib.Path(path).read_text()

# Deal with YAML header
header_text = mdtext.rsplit('---\n')[1]
header = yaml.safe_load('---\n' + header_text)

# Deal with Markdown Body
body = mdtext.rsplit('---\n')[2]

## Parse markdown body

In [7]:
# Set up the markdown parser
# to be honest, not fully sure what's going on here, see this issue: https://github.com/executablebooks/markdown-it-py/issues/164

mdit = MarkdownIt()
env = {}

# Set up tokens by parsing the md file
tokens = mdit.parse(body, env)

In [8]:
blocks = {}

block_count = 0

num_titles = 0

for x,t in enumerate(tokens):
     
    if t.tag == 'h1' and t.nesting == 1: # title
        blocks['title'] = [x,]
        num_titles += 1
        
    elif t.tag == 'h2' and t.nesting == 1:
        block_count += 1
        
        if block_count == 1:
            blocks['block{0}'.format(block_count)] = [x,]
        else:
            blocks['block{0}'.format(block_count-1)].append(x)
            blocks['block{0}'.format(block_count)] = [x,]

    #print(t,'\n')
    
# Add -1 to the end of the last block
blocks['block{0}'.format(block_count)].append(len(tokens))

# Assert statements (turn into tests!)
assert num_titles == 1, "I see {0} Level 1 Headers (#) in this file, there should only be one!".format(num_titles)
assert block_count > 1, "I see {0} Level 2 Headers (##) in this file, there should be at least 1".format(block_count -1)

# Add the end of the title block; # small hack
blocks['title'].append(blocks['block1'][0])

In [9]:
blocks

{'title': [0, 3], 'block1': [3, 9], 'block2': [9, 45], 'block3': [45, 81]}

In [10]:
## Process the blocks into markdown

body_parts = {}

part_counter = 0

for k,v in blocks.items():

    rendered_part = MDRenderer().render(tokens[v[0]:v[1]], mdit.options, env)
    
    if k == 'title':
        body_parts['title'] = rendered_part
    
    elif 'Rubric' in rendered_part:
        body_parts['Rubric'] = rendered_part

    elif 'Solution' in rendered_part:
        body_parts['Solution'] = rendered_part

    elif 'Comments' in rendered_part:
        body_parts['Comments'] = rendered_part
    
    else:
        part_counter +=1
        body_parts['part{0}'.format(part_counter)] = rendered_part

In [11]:
body_parts.keys()

dict_keys(['title', 'part1', 'part2', 'part3'])

In [14]:
header

{'title': 'Distance travelled',
 'type': 'multiple-choice',
 'author': 'Firas Moosvi',
 'template_version': 0.1,
 'source': 'original',
 'pl-options': {'allow-blank': True},
 'tags': ['kinematics', 'test'],
 'outcomes': ['LO.kinematics.2305', 'LO.kinematics.2304'],
 'assets': None,
 'server': 'import random\nimport pandas as pd\n\n# define the data dictionary\ndata = {"params":{},\n        "vars":{}}\n\n# define or load names/items/objects\nnames = pd.read_csv("data/names.csv")["Names"].tolist()\nmanual_vehicles = pd.read_csv("data/manual_vehicles.csv")["Manual Vehicles"].tolist()\n\n# store phrases etc\ndata["vars"]["name"] = random.choice(names)\ndata["vars"]["vehicle"] = random.choice(manual_vehicles)\ndata["vars"]["title"] = "Distance travelled"\ndata["vars"]["units"] = "m/s"\ndata["vars"]["digits_after_decimal"] = 2\n\n# define bounds of the variables\nv = random.randint(2,7)\nt = random.randint(5,10)\n\n# store the variables in the dictionary "params"\ndata["params"]["v"] = v\nda

In [15]:
print(body_parts['part1'])

## Question Text

{{ vars.name }} is traveling on {{ vars.vehicle }} at {{ params.v }} {{ vars.units }}.
How far does {{ vars.name }} travel in {{ params.t }} seconds, assuming they continue at the same velocity?



## Helper functions

In [16]:
def rounded(num, digits_after_decimal = 2):
    
    """
    Rounding as expected by normal sensible people.
    
    This needs to be heavily tested!!
    
    WARNING: This does not do sig figs yet!
    
    """

    # Solution copied from: https://stackoverflow.com/a/53329223

    from decimal import Decimal, getcontext, ROUND_HALF_UP

    round_context = getcontext()
    round_context.rounding = ROUND_HALF_UP
        
    tmp = Decimal(num).quantize(Decimal('1.'+'0'*digits_after_decimal))
    
    return str(tmp)

def dict_to_md(md_dict, remove_keys = [None,]):
    
    md_string = ""
    
    for k,v in md_dict.items():
        if k in remove_keys:
            continue
        else:
            md_string += md_dict[k]
            
    return md_string

## Demo of outputs

In [18]:
dict_to_md(body_parts)

'# {{ vars.title }}\n## Question Text\n\n{{ vars.name }} is traveling on {{ vars.vehicle }} at {{ params.v }} {{ vars.units }}.\nHow far does {{ vars.name }} travel in {{ params.t }} seconds, assuming they continue at the same velocity?\n## Part A\n\nHere is any instructions specific for this part\n\n### Answer Section\n\n- {{ params.ans1}} {{ vars.units}}\n- {{ params.ans2}} {{ vars.units}}\n- {{ params.ans3}} {{ vars.units}}\n- {{ params.ans4}} {{ vars.units}}\n- {{ params.ans5}} {{ vars.units}}\n## Part B\n\nMore instructions\n\n### Answer Section\n\n- {{ params.ans1}} {{ vars.units}}\n- {{ params.ans2}} {{ vars.units}}\n- {{ params.ans3}} {{ vars.units}}\n- {{ params.ans4}} {{ vars.units}}\n- {{ params.ans5}} {{ vars.units}}\n'